# # Visualización de respuestas

Importaciones

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

Conexión a la base de datos

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

Cargar las tablas de hechos del etl

In [3]:
df_hecho_novedades = pd.read_sql_table('hecho_novedades', etl_conn)
df_hecho_acumulating_servicios = pd.read_sql_table('hecho_acumulating_servicios', etl_conn)
df_hecho_servicios_dia = pd.read_sql_table('hecho_servicios_dia', etl_conn)
df_hecho_servicios_hora = pd.read_sql_table('hecho_servicios_hora', etl_conn)


Cargar las tablas de dimensiones del etl

In [4]:
df_dim_cliente = pd.read_sql_table('dim_cliente', etl_conn)
df_dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)
df_dim_hora = pd.read_sql_table('dim_hora', etl_conn)
df_dim_mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
df_dim_novedad = pd.read_sql_table('dim_novedad', etl_conn)
df_dim_sede = pd.read_sql_table('dim_sede', etl_conn)

# Verificar la cantidad de registros en las tablas de hechos

In [5]:
print('Fact Tables')
print('Hecho Novedades:', df_hecho_novedades.shape[0])
print('Hecho Acumulating Servicios:', df_hecho_acumulating_servicios.shape[0])
print('Hecho Servicios Dia:', df_hecho_servicios_dia.shape[0])
print('Hecho Servicios Hora:', df_hecho_servicios_hora.shape[0], '\n')


Fact Tables
Hecho Novedades: 5208
Hecho Acumulating Servicios: 26365
Hecho Servicios Dia: 12147
Hecho Servicios Hora: 3611 



# 1) En qué meses del año los clientes solicitan más servicios de mensajería

In [12]:
print('Meses con más solicitudes de servicios de mensajería')
response_1 = df_hecho_servicios_dia.merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')[['nombre_mes', 'mes', 'numero_servicios']]
response_1 = response_1.groupby(['nombre_mes', 'mes']).sum().sort_values(by='numero_servicios', ascending=False)
response_1

Meses con más solicitudes de servicios de mensajería


,,numero_servicios
nombre_mes,mes,
May,5,4409
July,7,4298
April,4,4095
August,8,4083
June,6,3896
March,3,3073
February,2,2201
January,1,262
December,12,22


# 2) ¿Cuáles son los días dónde más solicitudes hay?

In [13]:
print('Días con más solicitudes de servicios de mensajería')
response_2 = df_hecho_servicios_dia.merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')[['numero_servicios','nombre_dia']]
response_2 = response_2.groupby(['nombre_dia']).sum().sort_values(by='numero_servicios', ascending=False)
response_2


Días con más solicitudes de servicios de mensajería


,numero_servicios
nombre_dia,
Tuesday,5034
Friday,4889
Thursday,4782
Wednesday,4564
Monday,4012
Saturday,2294
Sunday,790


# 3) ¿A qué hora los mensajeros están más ocupados?

In [11]:

print('Horas con más solicitudes de servicios de mensajería')
response_3 = df_hecho_servicios_hora.merge(
    df_dim_hora,
    left_on='key_dim_hora_solicitud',
    right_on='key_dim_hora')[['hora', 'numero_servicios']]
response_3 = response_3.groupby(['hora']).sum().sort_values(by='numero_servicios', ascending=False)
response_3


Horas con más solicitudes de servicios de mensajería


,numero_servicios
hora,
09,3184
11,3086
10,2735
08,2474
15,2405
14,2369
16,1875
12,1604
13,1382


# 4) ¿Cuántos servicios ha solicitado cada cliente por mes?

In [16]:
print('Número de servicios solicitados por cliente y por mes')
response_4 = df_hecho_servicios_dia.merge(
    df_dim_cliente,
    left_on='key_dim_cliente',
    right_on='key_dim_cliente').merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')
response_4 = response_4.groupby(['cliente_id', 'nombre', 'nombre_mes']).agg({'numero_servicios': 'sum'}).reset_index()
response_4

Número de servicios solicitados por cliente y por mes


,cliente_id,nombre,nombre_mes,numero_servicios
0,2,Cliente 1,April,1
1,2,Cliente 1,February,4
2,2,Cliente 1,January,1
3,2,Cliente 1,June,1
4,2,Cliente 1,March,2


# 5) Mensajeros más eficientes (Los que más servicios prestan)

In [17]:
print('Mensajeros más eficientes')
response_5 = df_hecho_servicios_dia.merge(
    df_dim_mensajero,
    left_on='key_dim_mensajero',
    right_on='key_dim_mensajero')
response_5 = response_5.groupby(['mensajero_id', 'username']).agg({'numero_servicios': 'sum'}).sort_values(by='numero_servicios', ascending=False)
response_5

Mensajeros más eficientes


,,numero_servicios
mensajero_id,username,
30,JHONTROCHEZ,2402
29,Sebastian Acuña,1438
15,Juan Solano,1425
25,ANDRESGUTIERREZ,1392
16,Jan Sastre,1303
31,Hector Aquiles,1271
41,LUISCARDONA,1269
42,JPEDROZA,1214
22,Jairon Montes,1191


# 6) ¿Cuáles son las sedes que más servicios solicitan por cada cliente?

In [23]:
print('Sedes que más servicios solicitan por cada cliente')
response_6 = df_hecho_acumulating_servicios.merge(
    df_dim_sede,
    left_on='key_dim_sede',
    right_on='key_dim_sede').merge(
    df_dim_cliente,
    left_on='key_dim_cliente',
    right_on='key_dim_cliente').groupby(['cliente_id', 'nombre', 'sede_id', 'sede_nombre']).size().reset_index(name='numero_servicios')
response_6 = response_6.sort_values(by=['cliente_id', 'numero_servicios'], ascending=False)
response_6

Sedes que más servicios solicitan por cada cliente


,cliente_id,nombre,sede_id,sede_nombre,numero_servicios
40,27,MEDICOS Y SALUD,85,CIMEX,1
39,25,CALI SULUD Y VIDA,45,CAMBULOS,478
38,25,CALI SULUD Y VIDA,44,VALLE NORTE,474
37,24,CLINICA SAN RAFAEL DE OCCIDENTE,46,SAN BER NANDO,25
36,22,FUNDACION CLINICA INFANTIL LOS GRITONES,51,CLUB NOEL,57
34,12,CLINICA DEL SUR DE CALI,32,CLINICA FABILUX,995
35,12,CLINICA DEL SUR DE CALI,48,BATALLON DISPENSARIO,337
19,11,CARROS DEL PACIFICO (CHINA),15,PRINCIPAL NORTE / CHINA PACIFICO,6299
21,11,CARROS DEL PACIFICO (CHINA),17,BUSES Y CAMIONES -,4984
30,11,CARROS DEL PACIFICO (CHINA),26,PALMIRA BODEGA 20 /,1591


# 7) ¿Cuál es el tiempo promedio de entrega desde que se solicita el servicio hasta que se cierra el caso?

In [24]:
print('Tiempo promedio de entrega \n')
# make average of total_tiempo_minutos and total_tiempo_horas
response_7 = df_hecho_acumulating_servicios
avg_tiempo_minutos = round(response_7['total_tiempo_minutos'].mean(), 2)
avg_tiempo_horas = round(response_7['total_tiempo_horas'].mean(), 2)
print('Tiempo promedio en minutos:', avg_tiempo_minutos)
print('Tiempo promedio en horas:', avg_tiempo_horas, '\n')

Tiempo promedio de entrega 

Tiempo promedio en minutos: 404.42
Tiempo promedio en horas: 6.74 



# 8) Mostrar los tiempos de espera por cada fase del servicio: Iniciado, Con mensajero asignado, recogido en origen, Entregado en Destino, Cerrado. En que fase del servicio hay más demoras?

In [28]:
print('Tiempo de espera promedio por cada fase del servicio')
response_8 = df_hecho_acumulating_servicios

def get_avg_time(df, column):
    return round(df[column].mean(), 2)

response_8_mins = {
    'Asignacion minutos': get_avg_time(response_8, 'tiempo_minutos_asignacion'),
    'Recogido minutos': get_avg_time(response_8, 'tiempo_minutos_recogida'),
    'Entregado minutos': get_avg_time(response_8, 'tiempo_minutos_entrega'),
    'Cerrado minutos': get_avg_time(response_8, 'tiempo_minutos_cerrado'),
}

response_8_hours = {
    'Asignacion horas': get_avg_time(response_8, 'tiempo_horas_asignacion'),
    'Recogido horas': get_avg_time(response_8, 'tiempo_horas_recogida'),
    'Entregado horas': get_avg_time(response_8, 'tiempo_horas_entrega'),
    'Cerrado horas': get_avg_time(response_8, 'tiempo_horas_cerrado')
}

for key, value in response_8_mins.items():
    print(key, value)
for key, value in response_8_hours.items():
    print(key, value)
# get the max value
max_value_mins = max(response_8_mins, key=response_8_mins.get)
max_value_hours = max(response_8_hours, key=response_8_hours.get)
print('Fase con más demoras en minutos:', max_value_mins, " Tiempo: ", response_8_mins[max_value_mins])
print('Fase con más demoras en horas:', max_value_hours, " Tiempo: ", response_8_hours[max_value_hours])

Tiempo de espera promedio por cada fase del servicio
Asignacion minutos 147.41
Recogido minutos 85.76
Entregado minutos 87.11
Cerrado minutos 84.15
Asignacion horas 2.46
Recogido horas 1.43
Entregado horas 1.45
Cerrado horas 1.4
Fase con más demoras en minutos: Asignacion minutos  Tiempo:  147.41
Fase con más demoras en horas: Asignacion horas  Tiempo:  2.46


# 9) ¿Cuáles son las novedades que más se presentan durante la prestación del servicio?

In [29]:
print('Novedades que más se presentan durante la prestación del servicio')
response_9 = df_hecho_novedades.merge(
    df_dim_novedad,
    left_on='key_dim_novedad',
    right_on='key_dim_novedad').groupby(['tipo_novedad']).size().reset_index(name='numero_novedades')
response_9 = response_9.sort_values(by='numero_novedades', ascending=False)
response_9


Novedades que más se presentan durante la prestación del servicio


,tipo_novedad,numero_novedades
1,Novedades del servicio,3892
0,No puedo continuar,1316
